<a href="https://colab.research.google.com/github/majambo/rangeland/blob/main/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap
geemap.update_package()

In [1]:
import os
import ee
import geemap
import numpy as np
import pandas as pd
import geemap.colormaps as cm

In [2]:
m = geemap.Map(center=(4.18, 38.57), zoom=8)

In [3]:
# manage the date formating as per your requirements
# Mine is in format of YYYYMMdd
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [4]:
path_shapefile = "../OneDrive - CGIAR/Pasture/rangeland/rangeland1.shp"
region = geemap.shp_to_ee(path_shapefile)

In [28]:
# function to use with map
def clip_to_shapefile(img):
    return img.clip(region.geometry())

In [29]:
ndvi = ee.ImageCollection("MODIS/061/MOD13A2") \
            .filter(ee.Filter.date('2012-01-01', '2024-03-31')) \
            .select('NDVI')

In [30]:
sm = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001") \
              .filter(ee.Filter.date('2024-01-01', '2012-03-31')) \
              .select('SoilMoi00_10cm_tavg')
             #.filterBounds(region) \
              #.map(clip_to_shapefile)
#ee.ImageCollection("NASA/SMAP/SPL4SMGP/007") \

In [31]:
ptot = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
              .filter(ee.Filter.date('2024-01-01', '2024-03-31')) \
              .select('precipitation')


In [9]:
# Define a function to calculate the weekly average
def calculatebiWeeklyptot(imageCollection):
  daysInWeek = 16

  # Calculate the number of weeks in the collection
  weeks = ee.List.sequence(0, imageCollection.size().subtract(1).divide(daysInWeek).floor())

  # Function to calculate the weekly average
  def weeklyAverage(week):
    start = ee.Date(imageCollection.first().get('system:time_start')) \
      .advance(ee.Number(week).multiply(daysInWeek), 'day')
    end = start.advance(daysInWeek, 'day')

    weeklyImages = imageCollection \
      .filterDate(start, end);
      #.select('sm_surface'); 

    # Calculate the mean for the week
    #weeklyMean = weeklyImages.reduce(ee.Reducer.mean())
    weeklyMean = weeklyImages.reduce(ee.Reducer.sum())


    # Set the 'system:time_start' property to the start of the week
    startDate = ee.Date(start)
    newImage = weeklyMean.set('system:time_start', startDate.millis())

    return newImage

  return ee.ImageCollection.fromImages(weeks.map(weeklyAverage))

In [10]:
# Define a function to calculate the weekly average
def calculatebiWeeklysm(imageCollection):
  daysInWeek = 16

  # Calculate the number of weeks in the collection
  weeks = ee.List.sequence(0, imageCollection.size().subtract(1).divide(daysInWeek).floor())

  # Function to calculate the weekly average
  def weeklyAverage(week):
    start = ee.Date(imageCollection.first().get('system:time_start')) \
      .advance(ee.Number(week).multiply(daysInWeek), 'day')
    end = start.advance(daysInWeek, 'day')

    weeklyImages = imageCollection \
      .filterDate(start, end);
      #.select('sm_surface'); 

    # Calculate the mean for the week
    #weeklyMean = weeklyImages.reduce(ee.Reducer.mean())
    weeklyMean = weeklyImages.reduce(ee.Reducer.mean())


    # Set the 'system:time_start' property to the start of the week
    startDate = ee.Date(start)
    newImage = weeklyMean.set('system:time_start', startDate.millis())

    return newImage

  return ee.ImageCollection.fromImages(weeks.map(weeklyAverage))

In [32]:
# Calculate the weekly average
sm16 = calculatebiWeeklysm(sm)
ptot16 = calculatebiWeeklyptot(ptot)
#ndvi16 = calculateWeeklyAverage(ndvi)

In [12]:
points = pd.read_excel('../OneDrive - CGIAR/Pasture/rangeland/5km/grid_pointsv2.xlsx')
columns_to_round = ['X', 'Y']
points[columns_to_round] = points[columns_to_round].round(3)
points.head(2)

,X,Y,FID
0,39.410,3.271,1
1,39.445,3.261,2


In [13]:
features=[]
for index, row in points.iterrows():
#     print(dict(row))
#     construct the geometry from dataframe
    poi_geometry = ee.Geometry.Point([row['X'], row['Y']])
#     print(poi_geometry)
#     construct the attributes (properties) for each point
    poi_properties = dict(row)
#     construct feature combining geometry and properties
    poi_feature = ee.Feature(poi_geometry, poi_properties)
#     print(poi_feature)
    features.append(poi_feature)

# final Feature collection assembly
ee_fc = ee.FeatureCollection(features)
#ee_fc.getInfo()

In [ ]:
palette = cm.palettes.ndvi
ndvi_vis = {'min': 0.0, 'max': 9000.0, 'palette': palette}
palette = cm.palettes.ndwi
sm_vis = {'min': 0.0, 'max': 1.0, 'palette': palette}
palette = cm.palettes.ndwi
ppt_vis = {'min': 0.0, 'max': 200.0, 'palette': palette}

In [ ]:
m.addLayer(ndvi, ndvi_vis, 'NDVI')
m.addLayer(sm, sm_vis, 'soilmoisture')
m.addLayer(rainfall, ppt_vis, 'rainfall')
#m.addLayer(image.select(0), ndvi_vis, 'MODIS NDVI VIS')
#in_fc = geemap.shp_to_ee(points)
m.addLayer(ee_fc, {}, 'rangeland')
m.centerObject(ee_fc);

m

In [14]:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = ee_fc, # feature collection here
        scale = 10000 # Cell size of raster
    )
    return feature

In [33]:
sm1 = sm16.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()
ndvi1 = ndvi.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()
ptot1 = ptot16.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()

In [38]:
#sample_result = ptot1.first().getInfo()
#sample_result = sm1.first().getInfo()
sample_result = ndvi1.first().getInfo()
sample_result

{'type': 'Feature',
 'geometry': None,
 'id': '2012_01_01_0_0',
 'properties': {'FID': 1,
  'NDVI': 4991,
  'X': 39.41,
  'Y': 3.271,
  'date': 20120101}}

In [35]:
# extract the properties column from feature collection
# column order may not be as our sample data order
columns = list(sample_result['properties'].keys())
print(columns)

['FID', 'X', 'Y', 'date', 'precipitation_sum']


In [40]:
nested_list = ndvi1.reduceColumns(ee.Reducer.toList(len(columns)), columns).values().get(0)
data = nested_list.getInfo()

# dont forget we need to call the callback method "getInfo" to retrieve the data
df = pd.DataFrame(data, columns=columns)
# we obtain the data frame as per our demand
df

,FID,X,Y,date,precipitation_sum


In [ ]:
sm2 = df
sm2.to_csv('../OneDrive - CGIAR/Pasture/rangeland/5km/2024/data/sm.csv')

In [19]:
ndvi2 = df
ndvi2.to_csv('../OneDrive - CGIAR/Pasture/rangeland/5km/2024/data/ndvi.csv')

In [37]:
ptot2 = df
ptot2.to_csv('../OneDrive - CGIAR/Pasture/rangeland/5km/2024/data/ptot.csv')

In [ ]:
merged_df = pd.merge(ndvi2, sm2, on=['X', 'Y', 'date'], how='inner')
merged_df

In [ ]:
merged_df2 = pd.merge(merged_df, ptot2, on=['X', 'Y', 'date'], how='inner')
merged_df2

In [ ]:
selected_columns = ['X', 'Y', 'date', 'NDVI', 'sm_surface_mean', 'precipitation_sum']
result_df = merged_df2[selected_columns].loc[:, ~merged_df2[selected_columns].columns.duplicated()]
result_df

In [ ]:
# Define the new column names
new_column_names = ['lon', 'lat', 'date', 'ndvi', 'sm', 'ptot']

# Rename the columns
result_df.rename(columns=dict(zip(result_df.columns, new_column_names)), inplace=True)
result_df.head()

In [ ]:
result_df.to_csv('../OneDrive - CGIAR/Pasture/rangeland/5km/2024/combined_2024.csv')